<a href="https://colab.research.google.com/github/FLafage/Cours_Python/blob/main/Naissance_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Exercice sur la table des naissances journalières en 2024**



## **Upload de Google Drive :**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Import et chargement des données :**

In [ ]:
import pandas as pd

naissance_24 = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/FD_NAIS_2024.parquet')

naissance_24


,ANAIS,MNAIS,DEPNAIS,REGNAIS,SEXE,NBENF,AGEMERE,AGEXACTM,INDLNM,INDNATM,GAGPOM,GAGEXAPOM,DEPDOMM,REGDOMM
0,2024,01,69,84,2,1,26,25,1,1,40,35,01,84
1,2024,03,01,84,1,1,36,35,1,1,40,35,01,84
2,2024,04,69,84,2,1,28,27,1,1,25,25,01,84
3,2024,04,01,84,2,1,35,34,1,1,35,35,01,84
4,2024,04,69,84,2,1,33,32,1,1,35,35,01,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660782,2024,12,67,44,2,1,31,31,3,2,30,30,99,99
660783,2024,08,973,03,1,1,17,17,3,2,20,20,99,99
660784,2024,02,95,11,1,1,32,31,3,2,35,35,99,99
660785,2024,04,06,93,2,1,43,43,3,2,45,40,99,99


## **Type des variables :**

In [ ]:
naissance_24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660787 entries, 0 to 660786
Data columns (total 14 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   ANAIS      660787 non-null  object
 1   MNAIS      660787 non-null  object
 2   DEPNAIS    660787 non-null  object
 3   REGNAIS    660787 non-null  object
 4   SEXE       660787 non-null  object
 5   NBENF      660787 non-null  object
 6   AGEMERE    660787 non-null  object
 7   AGEXACTM   660787 non-null  object
 8   INDLNM     660787 non-null  object
 9   INDNATM    660787 non-null  object
 10  GAGPOM     660787 non-null  object
 11  GAGEXAPOM  660787 non-null  object
 12  DEPDOMM    660787 non-null  object
 13  REGDOMM    660787 non-null  object
dtypes: object(14)
memory usage: 70.6+ MB


## **Transformation des variables souhaitée en numérique :**

In [ ]:
naissance_24['NBENF'] = pd.to_numeric(naissance_24['NBENF'], errors='coerce')

naissance_24['AGEMERE'] = pd.to_numeric(naissance_24['AGEMERE'], errors='coerce')

## **Devoirs de François :**

Quel est le jour et le mois de l'année pour lequels le nombre de naissance est le plus élevé, pour l'ensemble de la France ?

In [ ]:
j_m_naissances = (naissance_24
    .groupby(['MNAIS'])
    .agg(total_enfants=('NBENF', 'sum'))
    .reset_index()
    .sort_values('total_enfants', ascending=False)
)

j_m_naissances

# Le mois avec le plus de naissance est le mois de Juillet avec 59390 naissances

,MNAIS,total_enfants
6,7,59390
9,10,58995
7,8,58501
4,5,58309
0,1,58006
8,9,56922
2,3,56454
11,12,55713
10,11,55050
3,4,54888


Est-ce que par département, il s'agit du même mois ?

In [ ]:
dep_m_naissances = (naissance_24
    .groupby(['DEPNAIS', 'MNAIS'])
    .agg(total_enfants=('NBENF', 'sum'))
    .reset_index()
    .sort_values(['DEPNAIS', 'total_enfants'], ascending=[True, False])
    .drop_duplicates(subset=['DEPNAIS'])
)

dep_m_naissances

,DEPNAIS,MNAIS,total_enfants
4,01,5,305
18,02,7,391
32,03,9,264
45,04,10,82
55,05,8,124
...,...,...,...
1161,971,10,379
1164,972,1,294
1187,973,12,654
1191,974,4,1129


Quel est la part de naissances gémellaires par département ?

In [ ]:
# On crée d'abord la colonne sur le dataframe existant
naissance_24['est_jumeau'] = (naissance_24['NBENF'] == 2)

# On fait l'agrégation
part_jumeaux_dep = (naissance_24
    .groupby('DEPNAIS')
    .agg(
        nb_jumeaux = ('est_jumeau', 'sum'),
        total_nais = ('NBENF', 'count')
    )
    .reset_index()
)

# On calcule la part à la fin
part_jumeaux_dep['part'] = (part_jumeaux_dep['nb_jumeaux'] / part_jumeaux_dep['total_nais']) * 100

# On sélectionne uniquement les variables d'intérêt
part_jumeaux_dep[['DEPNAIS', 'part']]

,DEPNAIS,part
0,01,2.180885
1,02,2.738386
2,03,2.051282
3,04,1.114827
4,05,1.502504
...,...,...
96,971,3.649433
97,972,3.834303
98,973,3.166034
99,974,3.296703


Quel est l'âge médian des mères ?

In [ ]:
age_mere = (naissance_24['AGEMERE']
            .median())

age_mere

# L'âge médian des mères est de 31 an

31.0